In [8]:
# Libraries
import pandas as pd
import geopandas as gpd
import statsmodels.formula.api as smf
import matplotlib.pyplot as plt

In [29]:
# Datasets
od = pd.read_csv("Project_dataset.csv")
samhsa = pd.read_csv("uscities_SAMHSA.csv", encoding="utf-8", encoding_errors="ignore")

In [30]:
od.head()
samhsa.head()

,Program Name,DBA,Street,City,State,zip code,Phone,Certification,FullCertification,County
0,"Serenity Health, LLC",NaN,780 W. Bel Air Ave Suite B,Aberdeen,MD,21001,(410) 273-1030,Certified,9/30/10,Harford
1,ARS of Aberdeen LLC,ARS of Aberdeen LLC,24-26 West Bel Air Ave.,Aberdeen,MD,21001,(410) 272-0230,Certified,3/18/16,Harford
2,"WCHS, Inc.",Grays Harbor Treatment Solutions,516 East 1st St.,Aberdeen,WA,98520,(877) 774-6565,Certified,5/4/20,Grays Harbor
3,Quinault Indian Nation,Quinault Wellness Center,511 W Heron St,Aberdeen,WA,98520,(564) 544-1950,Certified,8/10/23,Grays Harbor
4,"Abilene Absolute Recovery Solutions, OTP","Abilene Absolute Recovery Solutions, OTP [AARS...",429 North Judge Ely Blvd,Abilene,TX,79601,(325) 400-8586,Certified,4/27/22,Taylor


In [ ]:
# Group SAMHSA data by county
samhsa_county = samhsa.groupby("County").size().reset_index(name="facility_count")
samhsa_county

,County,facility count
0,Ada,3
1,Adams,7
2,Aguadilla,1
3,Aiken,1
4,Alachua,2
...,...,...
626,Yellowstone,1
627,Yolo,1
628,York,8
629,Yuba,2


In [28]:
merged = od.merge(samhsa_county, on="County", how="left")
# removing NA's with 0 in case we calculate %
merged["facility count"] = merged["facility count"].fillna(0)

merged

,State FIPS Code,County FIPS Code,State,FIPS Code,"State, County",County,median_household_income_2023,median_household_income_2022,median_household_income_2021,median_household_income_2020,...,opioid_dispensing_rate_2020,opioid_dispensing_rate_2021,opioid_dispensing_rate_2022,opioid_dispensing_rate_2023,drug_od_count_2019,drug_od_count_2020,drug_od_count_2021,drug_od_count_2022,drug_od_count_2023,facility count
0,51,720,VA,51720,"VA, Norton city",Norton city,42618,41298,38316,36004,...,441.0,390.7,374.9,403.9,0,0,0,0,0,0.0
1,51,775,VA,51775,"VA, Salem city",Salem city,63701,63676,60740,70349,...,377.7,396.4,379.3,342.2,0,0,0,0,0,0.0
2,51,690,VA,51690,"VA, Martinsville city",Martinsville city,44547,41500,38571,35715,...,315.7,302.4,234.3,176.5,0,0,0,0,0,0.0
3,21,193,KY,21193,"KY, Perry County",Perry County,42519,33585,31522,33712,...,284.7,280.5,270.0,264.5,7,18,35,22,25,0.0
4,42,93,PA,42093,"PA, Montour County",Montour County,73087,76229,68053,63341,...,260.2,246.4,241.7,230.0,1,1,5,3,4,0.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
3130,48,269,TX,48269,"TX, King County",King County,85720,87856,74890,78210,...,NaN,NaN,NaN,NaN,0,0,0,0,0,0.0
3131,48,301,TX,48301,"TX, Loving County",Loving County,86760,94716,100917,97491,...,NaN,NaN,NaN,NaN,0,0,0,0,0,0.0
3132,48,311,TX,48311,"TX, McMullen County",McMullen County,71112,67525,63120,45927,...,NaN,NaN,NaN,NaN,0,0,0,0,0,0.0
3133,2,63,AK,2063,"AK, Chugach Census Area",Chugach Census Area,95180,91356,88029,84063,...,NaN,NaN,NaN,NaN,0,0,0,0,0,0.0
